<a href="https://www.kaggle.com/code/syednahinhossain/bone-age-detection-mobilenet?scriptVersionId=99862603" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split

#loading training and testing datasets
df_train = pd.read_csv('../input/rsna-bone-age/boneage-training-dataset.csv')
df_test = pd.read_csv('../input/rsna-bone-age/boneage-test-dataset.csv')

#appending png file extension to id column for both training and testing datasets
df_train['id'] = df_train['id'].apply(lambda x: str(x)+'.png')
df_test['Case ID'] = df_test['Case ID'].apply(lambda x: str(x)+'.png')

#Feature Engineering
df_train['Sex'] = df_train['male'].apply(lambda x: '1' if x else '0')
del(df_train['male'])
df_test['id'] = df_test['Case ID']
df_test['Sex'] = df_test['Sex'].apply(lambda x: '1' if x=='M' else '0')
del(df_test['Case ID'])

#splitting train datasets into traininng and validation datasets
test_train_df, valid_df = train_test_split(df_train, test_size = 0.27, random_state = 0)
train_df, test_df = train_test_split(test_train_df, test_size = 0.2, random_state = 0)
test_df, orginal_test_df = train_test_split(test_df, test_size = 0.07, random_state = 0)

orginal_test_df.to_csv('original_test_data.csv')

In [ ]:
print(df_test.shape)
df_test.tail()

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D

#packages required for image preprocessing
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.metrics import mean_absolute_error
import tensorflow as tf
from keras.applications import MobileNet

#image_size = 256
image_size = 224

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
with tpu_strategy.scope():    
    #pretrain_model = ResNet50(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
    pretrain_model = MobileNet(input_shape=(image_size, image_size, 3), include_top=False, weights='imagenet')
    x = pretrain_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(1024,activation='relu')(x)
    x = Dense(512,activation='relu')(x)
    output_image = Dense(1,activation='linear')(x)

    #image_output = keras.Model(input=pretrain_model, output=output_image)
    gender_input = keras.Input(shape=(2,),name = 'gender')
    gender_concat =layers.concatenate([output_image,gender_input])
    output_gen = Dense(1,activation='linear')(gender_concat)
    #gender_model = keras.Model(inputs = gender_input,outputs = output_gen)


    model = keras.Model(inputs =[pretrain_model.input, gender_input],outputs=[output_gen])


    #keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

    model.compile(optimizer = 'adam', loss = 'mse',
                           metrics = ['mae'])



In [ ]:
# AUTO = tf.data.experimental.AUTOTUNE
# ignore_order = tf.data.Options()
# ignore_order.experimental_deterministic = False

# # On Kaggle you can also use KaggleDatasets().get_gcs_path() to obtain the GCS path of a Kaggle dataset
# from kaggle_datasets import KaggleDatasets
# filenames = KaggleDatasets().get_gcs_path('rsna-bone-age') # list files on GCS
# dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
# dataset = dataset.with_options(ignore_order)
# dataset = dataset.map(...) # TFRecord decoding here...

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
#from tensorflow.distribute import Strategy
with tpu_strategy.scope():
    save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
    #model = tf.keras.models.load_model('./model', options=load_locally) # loading in Tensorflow's "SavedModel" format
    weight_path="{}_mnet_weights.h5".format('Mobile_Net_bone_age')
    checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                                 save_best_only=True, mode='min', save_weights_only = True)


    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, verbose=1, mode='auto', min_delta=0.01, cooldown=3, min_lr=0.01)
    early = EarlyStopping(monitor="val_loss", 
                          mode="min", 
                          patience=10) # probably needs to be more patient
    callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
class CustomDataGen(tf.keras.utils.Sequence):
    
    def __init__(self, df, directory, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        self.n_sex = df[X_col['Sex']].nunique()
        #self.n_path = df[X_col['id']]
        self.directory = directory
    
    def __get_input(self, path, target_size):
        img_path = self.directory+path
        #print(img_path)
        image = tf.keras.preprocessing.image.load_img(img_path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy()

        return image_arr/255.
    
    def __get_gender(self, label, num_classes):
        return tf.keras.utils.to_categorical(label, num_classes=num_classes)
        
        
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['id']]
        sex_batch = batches[self.X_col['Sex']]

        bone_batch = batches[self.y_col['boneage']]

        X0_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])
        X1_batch = np.asarray([self.__get_gender(y, self.n_sex) for y in sex_batch])
        y_batch = np.asarray(bone_batch)

        return tuple([X0_batch,X1_batch]), y_batch
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __getitem__(self, index):

        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
target_size = (image_size, image_size, 3)
batch_size = 128
traingen = CustomDataGen(train_df,
                         directory="../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/",
                         X_col={'id':'id','Sex': 'Sex'},
                         y_col={'boneage': 'boneage'},
                         batch_size=batch_size, input_size=target_size)
validgen = CustomDataGen(valid_df,
                         directory="../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/",
                         X_col={'id':'id','Sex': 'Sex'},
                         y_col={'boneage': 'boneage'},
                         batch_size=batch_size, input_size=target_size)

In [ ]:
print(validgen[0])

In [ ]:
testgen = CustomDataGen(test_df,
                         directory="../input/rsna-bone-age/boneage-training-dataset/boneage-training-dataset/",
                         X_col={'id':'id','Sex': 'Sex'},
                         y_col={'boneage': 'boneage'},
                         batch_size=batch_size, input_size=target_size)

In [ ]:
class CustomDataGenTest(tf.keras.utils.Sequence):
    
    def __init__(self, df, directory, X_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True):
        
        self.df = df.copy()
        self.X_col = X_col
        #self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        self.n_sex = 2
        #self.n_path = df[X_col['id']]
        self.directory = directory
    
    def __get_input(self, path, target_size):
        img_path = self.directory+path
        #print(img_path)
        image = tf.keras.preprocessing.image.load_img(img_path)
        image_arr = tf.keras.preprocessing.image.img_to_array(image)

        image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy()

        return image_arr/255.
    
    def __get_gender(self, label, num_classes=2):
        return tf.keras.utils.to_categorical(label, num_classes=num_classes)
        
        
    def __get_data(self, batches):
        # Generates data containing batch_size samples

        path_batch = batches[self.X_col['id']]
        sex_batch = batches[self.X_col['Sex']]

        #bone_batch = batches[self.y_col['boneage']]

        X0_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])
        X1_batch = np.asarray([self.__get_gender(y, self.n_sex) for y in sex_batch])
        y = np.zeros(self.batch_size)

        return tuple([X0_batch,X1_batch]), y
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __getitem__(self, index):

        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__get_data(batches)        
        return X, y
    
    def __len__(self):
        return self.n // self.batch_size

In [ ]:
batch_size = 200
target_size = (image_size, image_size, 3)
test = CustomDataGenTest(df_test,
                         directory="../input/rsna-bone-age/boneage-test-dataset/boneage-test-dataset/",
                         X_col={'id':'id','Sex': 'Sex'},
                         batch_size=batch_size, input_size=target_size)

In [ ]:
print(len(test_df))
X,y = test[0]
print(len(y))

In [ ]:
train_df.head()

In [ ]:
model.load_weights('../input/bone-age-mnet-weightsh5/bone_age_mnet_weights.h5')

In [ ]:
# history_mobilenet = model.fit_generator(traingen,
#           validation_data=validgen,
#           epochs=50,callbacks = callbacks_list)

In [ ]:
df_test = pd.read_csv('../input/rsna-bone-age/boneage-test-dataset.csv')

print(df_test.tail())

In [ ]:
model.load_weights('../input/weight-file/bone_age_mnet_weights (2).h5')
pred = model.predict_generator(test, steps=len(test), verbose=1)

In [ ]:
result = pd.DataFrame(pred)
result.to_csv('result.csv')

In [ ]:
model.evaluate(testgen)